In [158]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras as kr
import random
import matplotlib as plt

In [96]:
with open('output/dfdata.ser', 'rb') as fl:
    main_data = pickle.load(fl)

In [97]:
rand_list = []
for i in range (100):
    rand_list.append(random.randint(0,len(main_data)))

In [98]:
for index in range(len(main_data['Home'])):
    main_data['Home'][index] = main_data['Home'][index].to_numpy()
    main_data['Away'][index] = main_data['Away'][index].to_numpy()

<ipython-input-98-ffbfcb272dae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['Home'][index] = main_data['Home'][index].to_numpy()
<ipython-input-98-ffbfcb272dae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['Away'][index] = main_data['Away'][index].to_numpy()


In [99]:
main_data['AWin'] = (np.less_equal(main_data["Hscore"], main_data['Ascore'])).astype(int)
main_data['HWin'] = (np.greater(main_data["Hscore"], main_data['Ascore'])).astype(int)

In [100]:
input_data1 = main_data['Home']
input_data2 = main_data['Away']
output_data = main_data[['HWin','AWin']]
output_data = main_data['HWin']
#output_data = main_data[['Hscore','Ascore']]

In [101]:
input_data1 = input_data1.to_numpy()
input_data2 = input_data2.to_numpy()
output_data = output_data.to_numpy()
for index in range(len(input_data1)):
    input_data1[index] = input_data1[index].flatten()
    input_data2[index] = input_data2[index].flatten()

    
test_data1 = []
test_data2 = []
test_out = []
for item in rand_list:
    test_data1.append(input_data1[item])
    test_data2.append(input_data2[item])
    test_out.append(output_data[item])

input_data1 = np.delete(input_data1,rand_list)
input_data2 = np.delete(input_data2,rand_list)
output_data = np.delete(output_data, rand_list)

input_data1 = input_data1.tolist()
input_data1 = tf.convert_to_tensor(input_data1,np.float32)
input_data2 = input_data2.tolist()
input_data2 = tf.convert_to_tensor(input_data2,np.float32)
output_data = tf.convert_to_tensor(output_data, np.float32)

In [104]:
test_data1 = tf.convert_to_tensor(test_data1,np.float32)
test_data2 = tf.convert_to_tensor(test_data2,np.float32)
test_out = tf.convert_to_tensor(test_out, np.float32)                                

In [105]:
type(test_data1)

tensorflow.python.framework.ops.EagerTensor

In [106]:
test_data1.shape

TensorShape([100, 902])

In [163]:
input1 = kr.layers.Input(shape=(902), name='I1')
dense1 = kr.layers.Dense(902, activation='relu')(input1)
dense1 = kr.layers.Dense(1804, activation='relu')(dense1)
dense10 = kr.layers.Dense(902, activation='relu',activity_regularizer=kr.regularizers.l1(0.01))(dense1)

input2 = kr.layers.Input(shape=(902), name='I2')
dense2 = kr.layers.Dense(902, activation='relu')(input2)
dense2 = kr.layers.Dense(1804, activation='relu')(dense2)
dense20 = kr.layers.Dense(902, activation='relu', activity_regularizer=kr.regularizers.l1(0.01))(dense2)

merge_one = kr.layers.concatenate([dense10, dense20])


dense3 = kr.layers.Dense(902, activation='relu')(merge_one)
dense3 = kr.layers.Dense(1804, activation='relu')(dense3)
# dense3 = kr.layers.Dropout(0.5)(dense3)
dense3 = kr.layers.Dense(902, activation='relu', activity_regularizer=kr.regularizers.l1(0.01))(dense3)
# dense3 = kr.layers.Dropout(0.5)(dense3)
dense3 = kr.layers.Dense(1, activation='sigmoid')(dense3)
# dense3 = kr.layers.Activation('relu')(dense3)


modef = kr.Model(inputs=[input1, input2], outputs= dense3)

In [164]:
modef.summary()

Model: "functional_48"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
I1 (InputLayer)                 [(None, 902)]        0                                            
__________________________________________________________________________________________________
I2 (InputLayer)                 [(None, 902)]        0                                            
__________________________________________________________________________________________________
dense_252 (Dense)               (None, 902)          814506      I1[0][0]                         
__________________________________________________________________________________________________
dense_255 (Dense)               (None, 902)          814506      I2[0][0]                         
______________________________________________________________________________________

In [165]:
optzr = kr.optimizers.Adamax(learning_rate=0.001)
modef.compile(loss='mean_squared_error', optimizer=optzr, metrics=["accuracy"])

In [166]:
history = modef.fit(x={'I1':input_data1, 'I2':input_data2}, y=output_data, epochs=10)

Epoch 1/10
69/69 [==============================] - 4s 52ms/step - loss: 4.8057 - accuracy: 0.5303
Epoch 2/10
69/69 [==============================] - 4s 54ms/step - loss: 0.2498 - accuracy: 0.5358
Epoch 3/10
69/69 [==============================] - 4s 52ms/step - loss: 0.2496 - accuracy: 0.5358
Epoch 4/10
69/69 [==============================] - 4s 58ms/step - loss: 0.2495 - accuracy: 0.5358
Epoch 5/10
69/69 [==============================] - 4s 56ms/step - loss: 0.2494 - accuracy: 0.5358
Epoch 6/10
69/69 [==============================] - 4s 54ms/step - loss: 0.2493 - accuracy: 0.5358
Epoch 7/10
69/69 [==============================] - 4s 55ms/step - loss: 0.2493 - accuracy: 0.5358
Epoch 8/10
69/69 [==============================] - 4s 59ms/step - loss: 0.2492 - accuracy: 0.5358
Epoch 9/10
69/69 [==============================] - 4s 58ms/step - loss: 0.2491 - accuracy: 0.5358
Epoch 10/10
69/69 [==============================] - 4s 57ms/step - loss: 0.2491 - accuracy: 0.5358


In [145]:
results = modef.evaluate(x={'I1':test_data1, 'I2':test_data2}, y=test_out)

4/4 [==============================] - 0s 8ms/step - loss: 0.2483 - accuracy: 0.5600
